### Imports

In [56]:
import torch
import torch.nn as nn

# импорты для готовых реализаций ResNet в torchvision
import torchvision
from torchvision import models

### Вспомогательные функции

Функции `conv3x3` и `conv1x1` являются вспомогательными функциями для создания сверточных слоев с ядрами размером 3x3 и 1x1 соответственно. Они упрощают создание блоков ResNet, делая код более читаемым и менее громоздким.

Разберем каждую функцию:

**`conv3x3(in_channels, out_channels, stride=1)`:**

* **`in_channels`:** Количество входных каналов.
* **`out_channels`:** Количество выходных каналов (количество фильтров в сверточном слое).
* **`stride=1`:** Шаг свертки. По умолчанию равен 1 (окно свертки сдвигается на один пиксель).
* **`kernel_size=(3, 3)`:** Размер ядра свертки - 3x3.
* **`padding=1`:**  Добавление отступа размером 1 пиксель по краям входного тензора.  Это обеспечивает сохранение пространственных размеров выходного тензора при `stride=1`.  `padding=1`  для ядра 3x3 - это "same" padding.
* **`bias=False`:**  Отключает использование bias (свободного члена) в сверточном слое. В ResNet bias часто отключают, так как Batch Normalization, применяемый после свертки, уже выполняет сдвиг активаций.


**`conv1x1(in_channels, out_channels, stride=1)`:**

* **`in_channels`:** Количество входных каналов.
* **`out_channels`:** Количество выходных каналов.
* **`stride=1`:** Шаг свертки.
* **`kernel_size=(1, 1)`:** Размер ядра свертки - 1x1.  Свертка 1x1  действует как линейное преобразование в каждом пикселе, позволяя изменять количество каналов (уменьшать или увеличивать) без изменения пространственных размеров.
* **`bias=False`:** Отключает использование bias.


**Зачем нужны эти функции?**

* **Читаемость кода:** Использование этих функций делает код более компактным и легким для понимания, скрывая детали создания сверточных слоев.
* **Повторное использование:**  Они позволяют избежать дублирования кода при создании множества сверточных слоев с одинаковыми параметрами.
* **Модификация:**  Если нужно изменить параметры сверточных слоев (например, добавить bias или изменить тип padding), достаточно изменить эти функции, а не все места, где они используются.


**В контексте ResNet:**

* `conv3x3` используется в `BasicBlock` для извлечения признаков.
* `conv1x1` используется в `Bottleneck` для уменьшения и увеличения количества каналов ("сжатие" и "расширение" в бутылочном горлышке), а также в `downsample` слоях для согласования размерностей.  Свертки 1x1 позволяют эффективно управлять количеством каналов, уменьшая вычислительную сложность модели.


В заключение, `conv3x3` и `conv1x1` - это удобные вспомогательные функции, которые делают код ResNet более чистым, модульным и легким для поддержки.  Они инкапсулируют логику создания сверточных слоев с определенными параметрами, что способствует лучшей организации кода.


In [33]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=(3,3),
        stride=stride,
        padding=1,
        bias=False
    )

def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=(1,1),
        stride=stride,
        bias=False
    )

### Basic block

Рализация класса для базового блока

Этот код определяет класс `BasicBlock`, который является базовым строительным блоком для архитектуры ResNet (Residual Network). Он реализует *residual connection* - ключевую концепцию ResNet, которая позволяет обучать очень глубокие нейронные сети, эффективно передавая градиенты и предотвращая проблему затухающих градиентов.

Разберем подробнее:

**1. `expansion = 1`:** Этот атрибут указывает, что количество выходных каналов блока равно количеству каналов, заданных в `out_channels`. В `BasicBlock` нет расширения каналов, в отличие от `Bottleneck` блока, где `expansion` обычно равно 4.

**2. `__init__(self, in_channels, out_channels, stride=1, downsample=None)`:**  Конструктор класса.

* **`in_channels`:** Количество входных каналов.
* **`out_channels`:** Количество выходных каналов (и каналов в промежуточных слоях, так как `expansion = 1`).
* **`stride`:** Шаг свертки. `stride=1` означает, что окно свертки сдвигается на один пиксель. `stride=2` уменьшит пространственные размеры карты признаков вдвое. Применяется в `self.conv1`.
* **`downsample`:** Опциональный аргумент. Если `stride > 1` или `in_channels != out_channels`, то `downsample` используется для преобразования входного тензора `x`, чтобы его размерность соответствовала размерности `out`. Это обычно реализуется с помощью свертки 1x1 и Batch Normalization.  Это необходимо для того, чтобы сложение в residual connection было корректным по размерностям.


**3. `forward(self, x)`:** Метод, определяющий прямой проход данных через блок.

* **`identity = x`:** Сохраняет входной тензор `x` для использования в residual connection.  Это "shortcut" ветка.
* **`out = self.conv1(x)` ... `out = self.bn2(out)`:** Два сверточных слоя 3x3 (`conv3x3`) с Batch Normalization (`bn1`, `bn2`) и ReLU активацией (`relu`) между ними. Это основная ветка обработки, где происходит извлечение признаков. Обратите внимание, что `stride` применяется только к первому сверточному слою `conv1`.
* **`if self.downsample is not None: identity = self.downsample(x)`:** Если `downsample` задан (т.е. если нужно изменить размерность входного тензора), он применяется к `x`, чтобы сделать его совместимым по размерности с `out` перед сложением.
* **`out += identity`:**  **Ключевой момент - residual connection.** Выход основной ветки (`out`) складывается с преобразованным или исходным входным тензором (`identity`). Это позволяет градиентам более эффективно распространяться во время обучения.
* **`out = self.relu(out)`:** ReLU активация применяется к результату суммирования.


**В итоге:** `BasicBlock` выполняет два сверточных слоя 3x3 и добавляет результат к исходному (или преобразованному) входу. Это позволяет сети изучать *остаточные* преобразования, которые могут быть более легкими для обучения, чем изучение полной трансформации. `downsample` используется для согласования размерностей при необходимости. `BasicBlock` является одним из основных строительных блоков ResNet и вносит существенный вклад в его способность к обучению очень глубоких сетей.  Он проще, чем `Bottleneck` блок, и используется в более "легких" вариантах ResNet (например, ResNet18, ResNet34).


**expansion**

В данном фрагменте кода `expansion = 1` является атрибутом класса `BasicBlock`.  Он используется для управления количеством выходных каналов в блоке ResNet.  В частности, он определяет, во сколько раз количество выходных каналов (`out_channels`) больше, чем количество каналов в промежуточных слоях блока.

В `BasicBlock`, `expansion = 1` означает, что количество выходных каналов равно количеству каналов, переданных в `out_channels` в конструкторе.  То есть, нет расширения количества каналов внутри блока.  

**Как это работает в контексте ResNet:**

ResNet (Residual Network) использует "бутылочные горлышки" (bottleneck blocks) и базовые блоки (basic blocks) в качестве строительных блоков.  Разница между ними заключается в количестве слоев и в том, как изменяется количество каналов.

* **Basic Block:**  Имеет два сверточных слоя 3x3. Количество каналов остается постоянным на протяжении всего блока и равно `out_channels`.  `expansion = 1` отражает это.

* **Bottleneck Block:** Имеет три сверточных слоя: 1x1, 3x3 и 1x1.  Первый слой 1x1 уменьшает количество каналов, средний слой 3x3 обрабатывает информацию с уменьшенным количеством каналов, а последний слой 1x1 восстанавливает количество каналов до исходного значения, умноженного на `expansion`.  В типичной реализации Bottleneck блока `expansion = 4`.


**Зачем нужен `expansion`:**

`expansion` позволяет гибко контролировать количество каналов в разных блоках ResNet.  Это важно для балансирования сложности модели и ее способности извлекать признаки.  Bottleneck блоки с `expansion > 1` позволяют уменьшить количество параметров в промежуточных слоях, что делает модель более эффективной с точки зрения вычислений.  В `BasicBlock`, где `expansion = 1`,  просто сохраняется одинаковое количество каналов на входе и выходе блока, сохраняя простоту структуры.


**В данном примере:**  `BasicBlock` с `expansion = 1`  - это самый простой тип блока в ResNet,  где количество каналов не меняется внутри блока.  Это означает, что `out_channels`  определяет количество каналов как на выходе блока, так и в промежуточных слоях.


In [34]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

### Bottleneck

Этот код определяет класс `Bottleneck`, который, как и `BasicBlock`, является строительным блоком для ResNet, но имеет более сложную структуру, предназначенную для более глубоких сетей.  Ключевое отличие - использование "бутылочного горлышка" (bottleneck) в архитектуре блока и `expansion = 4`.

**1. `expansion = 4`:**  Этот атрибут указывает, что количество выходных каналов блока в четыре раза больше значения `out_channels`, переданного в конструктор.  Расширение каналов происходит в последнем сверточном слое.

**2. `__init__(self, in_channels, out_channels, stride=1, downsample=None)`:** Конструктор определяет слои блока:

* **`self.conv1 = conv1x1(in_channels, out_channels)`:** Первый сверточный слой 1x1 **уменьшает** количество каналов с `in_channels` до `out_channels`. Это "сжатие" в бутылочном горлышке, которое уменьшает количество параметров и вычислений.
* **`self.conv2 = conv3x3(out_channels, out_channels, stride)`:** Второй сверточный слой 3x3 обрабатывает информацию с уменьшенным количеством каналов (`out_channels`).  Здесь применяется `stride`, если нужно уменьшить пространственное разрешение карты признаков.
* **`self.conv3 = conv1x1(out_channels, out_channels * self.expansion)`:** Третий сверточный слой 1x1 **расширяет** количество каналов до `out_channels * 4`. Это "расширение" в бутылочном горлышке, восстанавливающее размерность каналов, но с увеличенной в 4 раза шириной.
* **`self.bn1`, `self.bn2`, `self.bn3`:** Batch Normalization слои после каждого сверточного слоя для нормализации активаций и ускорения обучения.
* **`self.relu = nn.ReLU(inplace=True)`:** ReLU функция активации, применяемая после каждого слоя Batch Normalization, кроме финального.  `inplace=True` позволяет выполнять операцию без выделения дополнительной памяти.
* **`self.downsample`:**  Аналогично `BasicBlock`, используется для согласования размерностей, если `stride > 1` или `in_channels != out_channels * expansion`.  Позволяет корректно выполнить сложение в residual connection.


**3. `forward(self, x)`:** Метод прямого прохода:

* **`identity = x`:** Сохранение входного тензора для residual connection.
* Последовательность `conv -> bn -> relu` для первых двух сверточных слоев.
* `conv3 -> bn3`:  Третий сверточный слой с последующей Batch Normalization, но **без ReLU**.
* **`if self.downsample is not None: identity = self.downsample(x)`:** Применение `downsample` к входному тензору, если необходимо.
* **`out += identity`:** Residual connection - суммирование выхода bottleneck слоя с преобразованным или исходным входным тензором.
* **`out = self.relu(out)`:** Финальная ReLU активация после суммирования.  Обратите внимание, что ReLU применяется **после** сложения в residual connection.  


**В итоге:** `Bottleneck` блок использует структуру "бутылочного горлышка" 1x1 -> 3x3 -> 1x1 для более эффективной обработки информации и уменьшения количества параметров по сравнению с последовательностью слоев 3x3.  `expansion = 4` увеличивает количество выходных каналов. Residual connection обеспечивает эффективное обучение глубоких сетей. `Bottleneck` блоки используются в более глубоких вариантах ResNet (например, ResNet50, ResNet101, ResNet152).


In [40]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv1x1(in_channels, out_channels)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = conv3x3(out_channels, out_channels, stride)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = conv1x1(out_channels, out_channels*self.expansion)
        self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

### ResNet

Этот код реализует класс `MyResNet`, который позволяет создавать различные варианты архитектуры ResNet (ResNet18, ResNet34, ResNet50, ResNet101, ResNet152) в зависимости от входного параметра `name`. Код хорошо структурирован и следует общим принципам построения ResNet.

**Разберем подробнее:**

**1. `cfgs`:** Словарь, содержащий конфигурации для разных вариантов ResNet.  Ключ - имя модели (например, "resnet18"), значение - кортеж из двух элементов:
   *  Тип блока (`BasicBlock` или `Bottleneck`).
   *  Список, определяющий количество блоков в каждом из четырех слоев ResNet.  Например, `[2, 2, 2, 2]` для ResNet18 означает 2 блока в каждом слое.

**2. `__init__(self, name, num_classes=1000)`:** Конструктор класса.

* **`name`:** Строка, определяющая архитектуру ResNet (например, "resnet50").
* **`num_classes`:** Количество классов для классификации (по умолчанию 1000 для ImageNet).
* **`block, layers = self.cfgs[name]`:** Извлекает тип блока и количество блоков для заданной архитектуры из словаря `cfgs`.
* **`self.inplanes = 64`:** Инициализирует количество каналов на входе первого слоя.  Это значение будет обновляться в `make_layer`.
* **`self.conv1`, `self.bn1`, `self.relu`, `self.maxpool`:**  Первый сверточный слой 7x7 со `stride=2` и `padding=3`, Batch Normalization, ReLU активация и Max Pooling слой 3x3 со `stride=2` и `padding=1`.  Это начальные слои, общие для всех вариантов ResNet.
* **`self.layer1`, `self.layer2`, `self.layer3`, `self.layer4`:** Четыре основных слоя ResNet, каждый из которых состоит из нескольких блоков (`BasicBlock` или `Bottleneck`), определяемых `make_layer`.
* **`self.avgpool`:**  Average Pooling слой 7x7. **Важно:** как и обсуждали ранее, лучше использовать `nn.AdaptiveAvgPool2d((1, 1))` для входных изображений разных размеров.
* **`self.flatten`:** Преобразование многомерного тензора в одномерный вектор.
* **`self.fc`:** Полносвязный слой для классификации с количеством выходных нейронов, равным `num_classes`.


**3. `forward(self, x)`:**  Метод прямого прохода.  Последовательно применяет все слои к входному тензору `x` и возвращает результат `out` полносвязного слоя.

**4. `make_layer(self, block, out_channels, blocks, stride=1)`:**  Функция, создающая слой ResNet, состоящий из нескольких блоков.

* **`block`:** Тип блока (`BasicBlock` или `Bottleneck`).
* **`out_channels`:**  Желаемое количество выходных каналов для блока.  Обратите внимание, что реальное количество выходных каналов блока будет `out_channels * block.expansion`.
* **`blocks`:** Количество блоков в слое.
* **`stride`:** Шаг свертки для первого блока в слое.  
* **`downsample`:** Создает `downsample` слой (свертка 1x1 + Batch Normalization), если необходимо согласовать размерности между входом слоя и выходом первого блока.
* Цикл создает нужное количество блоков и добавляет их в список `layers`.  **Важно:** `downsample` применяется только к первому блоку в слое.
* Возвращает `nn.Sequential(*layers)` - контейнер, который последовательно применяет все блоки в слое.  


**В целом:**  Код представляет собой хорошую реализацию ResNet, позволяющую создавать разные варианты архитектуры.  Однако, рекомендуется заменить `nn.AvgPool2d((7,7))` на `nn.AdaptiveAvgPool2d((1, 1))` для большей гибкости.  Также стоит обратить внимание на применение `downsample` только к первому блоку в слое и убедиться, что это соответствует выбранной архитектуре ResNet.


In [47]:
class MyResNet(nn.Module):
    cfgs = {
        "resnet18":(BasicBlock,[2, 2, 2, 2]),
        "resnet34":(BasicBlock,[3, 4, 6, 3]),
        "resnet50":(Bottleneck,[3, 4, 6, 3]),
        "resnet101":(Bottleneck,[3, 4, 23, 3]),
        "resnet152":(Bottleneck,[3, 8, 36, 3])
    }

    def __init__(self, name, num_classes=1000):
        super().__init__()
        block, layers = self.cfgs[name]

        self.inplanes = 64

        self.conv1 = nn.Conv2d(3, self.inplanes, (7,7), stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((3, 3), stride=2, padding=1)
        self.layer1 = self.make_layer(block, 64, layers[0])
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d((7,7))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = self.flatten(x)
        out = self.fc(x)

        return out

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []

        downsample = None
        if stride != 1 or self.inplanes != out_channels * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, out_channels*block.expansion, stride),
                nn.BatchNorm2d(out_channels*block.expansion)
            )

        layers.append(
            block(self.inplanes, out_channels, stride, downsample)
        )

        self.inplanes = out_channels * block.expansion
        
        for _ in range(1, blocks):
            layers.append(
                block(self.inplanes, out_channels)
            )
            
        return nn.Sequential(*layers)

In [52]:
# создадим модель
model = MyResNet('resnet18')
model

MyResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=(3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=T

In [53]:
# проверим работу созданной модели
inp = torch.rand([1, 3, 224, 224], dtype=torch.float32)
pred = model(inp)
print(pred.shape)

torch.Size([1, 1000])


### Готовые архитектуры ResNet в Torchvision

В данной части все по сути аналогично VGG (см. предыдущий ноутбук)

In [57]:
# создадим готовую модель из модуля
model = models.resnet50()

In [58]:
# создадим модель с предобученными весами
model = models.resnet50(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /home/talium/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth

00%|██████████████████████████████████████| 97.8M/97.8M [00:01<00:00, 54.4MB/s]

In [60]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 